# Exercícios Aula 6 - Thiago Garbin

In [11]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError

# Exercício 1 – Scraping da loja virtual

In [12]:
# (https://www.cea.com.br/moda?filters=[]&pageNumber=1)
# Na URL acima temos um catálogo de vendas online de roupas, acessórios, etc.
# Vamos capturar as informações dos produtos que estão disponíveis no site e uma imagem de cada um
# desses itens.
# Objetivo final é obter um data frame que possua as seguintes informações:
# • Nome do produto;
# • Preço;
# • Link que direcione para a página com os detalhes do produto;
# • Nome da foto (referência para a foto que será capturada via download).
# Observe que no final da página há um botão que navega para as páginas seguintes. Neste exercício,
# queremos buscar todo o conteúdo disponível até a página 20.
# A URL fornecida (acima) lista apenas os produtos da primeira página. Portanto será preciso modifica-la a
# cada iteração, para buscar a próxima página.
# Obs.: nessa atividade vamos usar o navegador Google Chrome.

### Passo-a-passo

In [13]:
# 1. Abra o link no navegador e selecione a ferramenta de desenvolvedores (Menu > More Tools >
# Developer Tools). Deixe-a aberta para as consultas ao longo do exercício.

In [14]:
# 2. Estabeleça a conexão com o site e crie o objeto html. Feito isso, crie o objeto BeautifulSoup
# passando como parâmetro o html e indicando ‘html-parser’. Opcionalmente, antes de criar o
# objeto BeautifulSoup aplique o método de tratamento de strings que vimos em aula.

# O link passado originalmente retornava no web scraping apenas os resultados da primeira página,
# mesmo quando era alterado o número da página ao final. Conversando com colegas, um deles descobriu
# o link interno utilizado pelo javascript para carregar as diferentes páginas, o qual será então
# utilizado no trabalho.

# Depois de já enviado o trabalho, o professor postou no exemplo de aula como seria possível navegar
# pelas páginas e obter as informações com o link original, com o uso de selenium, que cria um webdriver
# faz um get das páginas. Ou seja, é como se estivesse usando o navegador.

url = 'https://www.cea.com.br/buscapagina?fq=H%3a11633&O='\
'OrderByReleaseDateDESC&PS=48&sl=267cfeec-2b17-4122-9f04-c7abf8e5a82d&cc=1&sm=0&PageNumber=1'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '\
           '(KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

req = Request(url, headers = headers)
response = urlopen(req)
html = response.read()

In [15]:
html = html.decode('utf-8')
type(html)

str

In [16]:
def prep_html(text):
    return " ".join(html.split()).replace('> <','><')

In [17]:
html = prep_html(html)

In [18]:
soup = BeautifulSoup(html, 'html.parser')

print(soup.prettify()[:2000])

<html>
 <head>
  <meta content="NOINDEX, FOLLOW" name="ROBOTS"/>
 </head>
 <body>
  <div class="res-prateleira-vitrine-v3 n1colunas">
   <ul>
    <li class="moda-feminina--roupas-blusas-vestidos-saias-jaquetas-|-c-a last" layout="267cfeec-2b17-4122-9f04-c7abf8e5a82d">
     <div class="product-image" data-productid="3349712">
      <figure>
       <a class="product-image_link" href="https://www.cea.com.br/camiseta-feminina-manga-curta--tropicaya--decote-redondo-preta-9990245-preto/p" title='Camiseta Feminina Manga Curta "Tropicaya" Decote Redondo Preta'>
        <script class="product-image_main" type="text/html">
         <img src="https://cea.vteximg.com.br/arquivos/ids/34958501-463-556/Camiseta-Feminina-Manga-Curta--Tropicaya--Decote-Redondo-Preta-9990245-Preto_1.jpg?v=637520141442830000" width="463" height="556" alt="Camiseta-Feminina-Manga-Curta--Tropicaya--Decote-Redondo-Preta-9990245-Preto_1" id="" />
        </script>
        <noscript>
         <img alt="Camiseta-Feminina-Manga

In [19]:
# 3. Navegue no objeto soup para encontrar as informações (nome do produto, preço e link).
# Encontre também uma forma de localizar a imagem do produto.

In [20]:
soup.find('div', attrs={'class':'product-details'})

<div class="product-details" data-productid="3349712"><a class="product-details_name" href="https://www.cea.com.br/camiseta-feminina-manga-curta--tropicaya--decote-redondo-preta-9990245-preto/p" title='Camiseta Feminina Manga Curta "Tropicaya" Decote Redondo Preta'><strong>Camiseta Feminina Manga Curta "Tropicaya" Decote Redondo Preta </strong></a><div class="product-flag-discount-list" style="display:none;"><p class="flag id-11671--promo-combo-blusas-femininas">id-11671--promo-Combo-blusas-femininas</p></div><div class="product-details_prices product-details_prices_ab"><a href="https://www.cea.com.br/camiseta-feminina-manga-curta--tropicaya--decote-redondo-preta-9990245-preto/p" title='Camiseta Feminina Manga Curta "Tropicaya" Decote Redondo Preta'><span class="product-details_prices--parcel price-highlight"> em até <br/><strong>3x</strong> de <strong>R$ 8,66</strong><br/><span class="price-disclaimer">no cartão C&amp;A <strong>sem juros</strong></span></span><strong class="product-de

In [21]:
soup.find('div', attrs={'class':'product-details'}).get_text()

'Camiseta Feminina Manga Curta "Tropicaya" Decote Redondo Preta\xa0id-11671--promo-Combo-blusas-femininas em até 3x de R$ 8,66no cartão C&A sem jurosou R$ 25,99 à vista'

In [22]:
soup.find('a', attrs={'class':'product-details_name'}).text[:-1]

'Camiseta Feminina Manga Curta "Tropicaya" Decote Redondo Preta'

In [23]:
soup.find('strong', attrs={'class':'product-details_prices--to'}).find('span').text

'R$ 25,99'

In [24]:
soup.find('div', attrs={'class':'product-flags'}).a.get('href').replace('https://','')

'www.cea.com.br/camiseta-feminina-manga-curta--tropicaya--decote-redondo-preta-9990245-preto/p'

In [25]:
soup.find('div', attrs={'class':'product-image'}).img.get('src').split('?v=')[0]

'https://cea.vteximg.com.br/arquivos/ids/34958501-463-556/Camiseta-Feminina-Manga-Curta--Tropicaya--Decote-Redondo-Preta-9990245-Preto_1.jpg'

In [26]:
# 4. Da mesma forma como fizemos em aula, crie um laço que execute a extração sobre cada um dos
# itens da página, incluindo o download de cada imagem. Por fora desse laço, crie outro que faça a
# iteração das páginas (de 1 a 20).

# Dicas:
# • O nome da foto que será mantido no data frame final e no arquivo local pode ser uma número
# incremental (aproveite o laço de repetição externo para formar esse nome).
# • Alguns valores de atributos do HTML podem apresentar vários espaços no final da string.
# Atente-se, pois em alguns casos será preciso retirar esses espaços para que o método find e
# findAll funcione corretamente.

anuncios=[]

nro_foto = 0

paginas = 20

for pagina in range(1, paginas+1):

    url = 'https://www.cea.com.br/buscapagina?fq=H%3a11633&O='\
    'OrderByReleaseDateDESC&PS=48&sl=267cfeec-2b17-4122-9f04-c7abf8e5a82d&cc=1&sm=0&PageNumber='\
    + str(pagina)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '\
               '(KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read()

    html = html.decode('utf-8')

    html = prep_html(html)
    
    soup = BeautifulSoup(html, 'html.parser')    

    for i in soup.findAll('li', attrs={'layout':'267cfeec-2b17-4122-9f04-c7abf8e5a82d'}):
        
        produto = i.find('a', attrs={'class':'product-details_name'}).text[:-1]
        preco = i.find('strong', attrs={'class':'product-details_prices--to'}).span.text
        link = i.find('div', attrs={'class':'product-flags'}).a.get('href').replace('https://','')
        foto= i.find('div', attrs={'class':'product-image'}).img.get('src').split('?v=')[0]

        nro_foto += 1

        anuncio=[]

        anuncio.append(produto)
        anuncio.append(preco)
        anuncio.append(link)
        anuncio.append('foto_item_' + str(nro_foto) + '.jpg')

        urlretrieve(foto, './fotos/' + 'foto_item_' + str(nro_foto) + '.jpg')

        anuncios.append(anuncio)

In [27]:
df = pd.DataFrame(anuncios, columns=['Produto','Preco', 'Link', 'Nome_Foto'])

df

,Produto,Preco,Link,Nome_Foto
0,"Camiseta Feminina Manga Curta ""Tropicaya"" Deco...","R$ 25,99",www.cea.com.br/camiseta-feminina-manga-curta--...,foto_item_1.jpg
1,Short de Moletom Feminino Cintura Alta com Rec...,"R$ 59,99",www.cea.com.br/short-de-moletom-feminino-cintu...,foto_item_2.jpg
2,Calça Feminina Carrot Cintura Alta Alfaiataria...,"R$ 139,99",www.cea.com.br/calca-feminina-carrot-cintura-a...,foto_item_3.jpg
3,Body Infantil Manga Curta Listrado com Babados...,"R$ 35,99",www.cea.com.br/body-infantil-manga-curta-listr...,foto_item_4.jpg
4,Macaquinho Infantil Estampado de Corações Alça...,"R$ 49,99",www.cea.com.br/macaquinho-infantil-estampado-d...,foto_item_5.jpg
...,...,...,...,...
955,Calça Legging Infantil com Glitter Lilás,"R$ 29,99",www.cea.com.br/calca-legging-infantil-com-glit...,foto_item_956.jpg
956,Calça Legging Infantil com Glitter Verde,"R$ 29,99",www.cea.com.br/calca-legging-infantil-com-glit...,foto_item_957.jpg
957,Short Saia Infantil Listrado com Corações Off ...,"R$ 39,99",www.cea.com.br/short-saia-infantil-listrado-co...,foto_item_958.jpg
958,Short Saia Infantil Estampado Arco-Íris com Gl...,"R$ 39,99",www.cea.com.br/short-saia-infantil-estampado-a...,foto_item_959.jpg


In [28]:
df['Produto'][456]

'Camisa Masculina Tradicional Estampada de Folhagem com Linho Manga Curta Branca'

In [29]:
df['Preco'][456]

'R$ 119,99'

In [30]:
df['Link'][456]

'www.cea.com.br/camisa-masculina-tradicional-estampada-de-folhagem-com-linho-manga-curta-branca-9982807-branco/p'

In [31]:
df['Nome_Foto'][456]

'foto_item_457.jpg'